# Employee database project


- SQL is a delclarative language, it is not prefered to use in data manipulation, Exploratory Data Analysis(EDA). 


- For that we need some more powerful language such as python. Because python has many libraries like `numpy`, `scipy`, `pandas`, `matplotlib`, which helps you explore data, do statistical analysis and plot the data.

So in this lesson we will see how can we connect SQL with python language. using **MYSQL connector**.

## Libraries
To install mysql connector in your anaconda environment execute the following command in your jupyter notebook cell.

``` python
    !pip install mysql-connector-python 
```

As in every python project, the first thing we do is import all the required libraries. So let's import all the libraries as given below.

In [1]:
!pip install mysql-connector-python

In [4]:
# import libraries
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [5]:
# create connection with server

def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection


- `host_name`: MySQL server has a host name which is usually `localhost`.
- `user_name`: Similarly host name MYSQL server has username mostly it is `root`.
- `user_password`: user_password is a database password which adds an layer of security to your MYSQL database.


You can use this function can be used in future projects too.

In [6]:
connection = create_server_connection("localhost", "root", "mlwithrish30")

MySQL Database connection successful


## Creating a Database



In [7]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")


- `connection`: It is a connection object we will get from the `create_server_connection()` function.
- `query`: It is a SQL query which will be passed in the function call given below.

We use the `cursor method` on our `connection object` to create a `cursor object`. `MYSQL Connector` uses OOPs concepts there are going to many objects and methods used in this lesson.


Imagine this cursor method as the cursor we get in the MYSQL command line as shown below:
![](images/mysql_cursor.png)

In [8]:
create_database_query = 'CREATE DATABASE company'
create_database(connection, create_database_query)

Database created successfully


## Connecting to the Database

In [9]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name  
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

The function which we have written above named `create_db_connection` is same as the function which we have used in `create_server_connection` function with just one extra agrument that is `db_name`. The function makes a conncetion with the database whose name you have provided in the function call. 

In [10]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

`execute_query` function is going to be our primary function for many operations like
- creating tables
- establish relationships between those tables 
- populate tables etc.

## Creating Tables

Now we have created all the python functions which will be required. We will now move to the SQL part of wrting SQL queries. So let's begin by creating our first table of employees. 

In [11]:
# use appropriate name

create_employees_table = """
CREATE TABLE employees (
    emp_no INT(11) NOT NULL AUTO_INCREMENT,
    first_name VARCHAR(15) NOT NULL,
    last_name VARCHAR(15) NOT NULL,
    hire_date VARCHAR(15) NOT NULL,
    gender ENUM('M','F') NOT NULL,
    birth_date VARCHAR(15) NOT NULL,
    PRIMARY KEY (emp_no)
    );
"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company') # Connect to the Database
execute_query(connection, create_employees_table) # Execute our defined query

MySQL Database connection successful
Query successful


Running the above code gives us our success messages. We can also verify this in the MySQL Workbech:

Great! Now let's create rest of the tables.

In [12]:
create_departments_table = """
CREATE TABLE departments (
    dept_no VARCHAR(10) NOT NULL,
    dept_name VARCHAR(40) NOT NULL,
    PRIMARY KEY (dept_no), 
    UNIQUE KEY dept_name(dept_name)
    );
"""
create_salaries_table = """
CREATE TABLE salaries (
    emp_no INT(11) NOT NULL,
    salary INT(11) NOT NULL,
    from_date VARCHAR(15) NOT NULL,
    to_date VARCHAR(15) NOT NULL,
    PRIMARY KEY (emp_no,from_date), 
    KEY emp_no (emp_no),
    CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no)
        REFERENCES employees (emp_no) ON DELETE CASCADE
    );
"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
execute_query(connection, create_departments_table)
execute_query(connection, create_salaries_table)

MySQL Database connection successful
Query successful
Query successful


In [13]:
create_deptemp_table = """
CREATE TABLE dept_emp (
    emp_no INT(11) NOT NULL,
    dept_no VARCHAR(10) NOT NULL,
    from_date VARCHAR(15) NOT NULL,
    to_date VARCHAR(15) NOT NULL,
    PRIMARY KEY (emp_no, dept_no), 
    KEY emp_no (emp_no),
    KEY dept_no (dept_no),
    CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no)
        REFERENCES employees (emp_no) ON DELETE CASCADE,
    CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no)
        REFERENCES departments (dept_no) ON DELETE CASCADE
);
"""

create_dept_manager_table = """
CREATE TABLE dept_manager (
    emp_no INT(11) NOT NULL,
    dept_no VARCHAR(10) NOT NULL,
    from_date VARCHAR(15) NOT NULL,
    to_date VARCHAR(15) NOT NULL,
    PRIMARY KEY (emp_no, dept_no),
    KEY emp_no (emp_no),
    KEY dept_no (dept_no),
    CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) 
        REFERENCES employees (emp_no) ON DELETE CASCADE,
    CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no)
        REFERENCES departments (dept_no) ON DELETE CASCADE
    );
"""

create_titles_table ="""
CREATE TABLE titles (
    emp_no INT(11) NOT NULL,
    title VARCHAR(50) NOT NULL,
    from_date VARCHAR(15) NOT NULL,
    to_date VARCHAR(15) DEFAULT NULL,
    PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),
    CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)
        REFERENCES employees (emp_no) ON DELETE CASCADE
    );
"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
execute_query(connection, create_deptemp_table)
execute_query(connection, create_dept_manager_table)
execute_query(connection, create_titles_table)

MySQL Database connection successful
Query successful
Query successful
Query successful


## Inserting values to the tables

The next step is to add some records to the tables. Again we use execute_query to feed our existing SQL commands into the Server. Let's again start with the Employee table.

In [16]:
pop_employee = """
INSERT INTO employees VALUES
(1, 'Raju', 'Rathi', '1999-02-20', 'M', '1977-06-14'),
(2, 'Shyam', 'Naik', '2007-11-08', 'M', '1985-11-01'), 
(3, 'Baburao', 'Apte', '2014-06-11', 'M', '1984-02-01'),
(4, 'Anjali', 'Pande', '2000-08-23', 'F', '1978-05-19'),
(5, 'Abhilasha', 'Mohite', '2020-03-30', 'F', '1994-01-17'),
(6, 'Suresh', 'Kadam', '1999-02-21', 'M', '1977-08-21'),
(7, 'Manish', 'Joshi', '2014-06-11', 'M', '1992-09-29'),
(8, 'Radha',  'Marathe', '2020-03-29', 'F', '1988-11-22')
"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
execute_query(connection, pop_employee)

MySQL Database connection successful
Query successful


Verify on workbench!

In [19]:
from datetime import datetime,date

In [17]:
pop_dept = """
INSERT INTO departments VALUES
('HR01', 'Human Resources'),
('SFT01', 'S.Development'),
('SFT02', 'S.Testing'),
('MKT01', 'Marketing')
"""


pop_manager = """
INSERT INTO dept_manager VALUES
(2, 'SFT01','2008-12-20','2012-03-19'),
(3, 'MKT01', '2014-06-11', '2020-10-04'),
(6, 'SFT02', '2002-04-07','2004-11-10'),
(8, 'HR01', '2020-03-29','2022-03-03')
""" 
    

pop_title = """
INSERT INTO titles VALUES
(1,'Sr.Soft.Developer','2002-12-06', '2007-07-18'),
(2,'Soft.Developer Mngr.','2008-12-20', '2012-03-19'),
(3,'Marketing Mngr','2014-06-11', '2020-10-04'),
(4,'Soft.Tester','2000-08-23', '2005-12-03'),
(5,'HR Staff','2020-03-30','2021-10-09'),
(6,'Soft.Tester Mngr','2002-12-06','2004-11-10'),
(7,'Marketing Staff','2014-06-11','2016-12-04'),
(8,'HR Manager','2020-03-29','2022-03-03')
"""
    
    
connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')

execute_query(connection, pop_dept)
execute_query(connection, pop_manager)
execute_query(connection, pop_title)

MySQL Database connection successful
Query successful
Query successful
Query successful


## Inserting values in the form of Lists

We saw when inserting values to our tables that we can use the **SQL INSERT** command in our `execute_query` function.

List method is also more secure if our database is open to our users at any point, as it helps to prevent against [SQL Injection attacks](https://portswigger.net/web-security/sql-injection), which can damage or even destroy our whole database.

To do this, we will write a function using the `.executemany()` method, instead of the simpler `execute()` method we have been using thus far.

In [14]:
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [20]:
pop_salary = """INSERT INTO salaries (emp_no, salary, from_date, to_date) VALUES (%s, %s, %s, %s)"""

today = datetime.now().date()

sal_data = [(1, 1234567, '1999-02-20', today.isoformat()),
(2, 2345678, '2007-11-08', today.isoformat()),
(3, 3456789, '2014-06-11', today.isoformat()),
(4, 4567890, '2000-08-23', today.isoformat()),
(5, 5678901, '2020-03-30', today.isoformat()),
(6, 6789012, '1999-02-21', today.isoformat()),
(7, 7890123, '2014-06-11', today.isoformat()),
(8, 8901234, '2020-03-29', today.isoformat())]
    
pop_deptemp = """INSERT INTO dept_emp (emp_no, dept_no, from_date, to_date) VALUES (%s, %s, %s, %s)"""

deptemp_data = [(1, 'SFT01','1999-02-20', today.isoformat()),
(2, 'SFT01','2007-11-08',today.isoformat()),
(3, 'MKT01','2014-06-11',today.isoformat()),
(4, 'SFT02','2000-08-23',today.isoformat()),
(5, 'HR01','2020-03-30',today.isoformat()),
(6, 'SFT02','1999-02-21',today.isoformat()),
(7, 'MKT01','2014-06-11',today.isoformat()),
(8, 'HR01','2020-03-29',today.isoformat())]

In [21]:
connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
execute_list_query(connection, pop_salary, sal_data)
execute_list_query(connection, pop_deptemp, deptemp_data)

MySQL Database connection successful
Query successful
Query successful


Amazing! Now we have created a database complete with relations, constraints and records in MySQL, using nothing but Python commands.

## Reading Data

For this, we will need one more function, this time using `cursor.fetchall()` instead of `cursor.commit()`. With this function, we are reading data from the database and will not be making any changes.

In [22]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

In [23]:
q1 = """
SELECT *
FROM employees;
"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
results = read_query(connection, q1)

results

MySQL Database connection successful


[(1, 'Raju', 'Rathi', '1999-02-20', 'M', '1977-06-14'),
 (2, 'Shyam', 'Naik', '2007-11-08', 'M', '1985-11-01'),
 (3, 'Baburao', 'Apte', '2014-06-11', 'M', '1984-02-01'),
 (4, 'Anjali', 'Pande', '2000-08-23', 'F', '1978-05-19'),
 (5, 'Abhilasha', 'Mohite', '2020-03-30', 'F', '1994-01-17'),
 (6, 'Suresh', 'Kadam', '1999-02-21', 'M', '1977-08-21'),
 (7, 'Manish', 'Joshi', '2014-06-11', 'M', '1992-09-29'),
 (8, 'Radha', 'Marathe', '2020-03-29', 'F', '1988-11-22')]

In [24]:
for result in results:
    print(result)

(1, 'Raju', 'Rathi', '1999-02-20', 'M', '1977-06-14')
(2, 'Shyam', 'Naik', '2007-11-08', 'M', '1985-11-01')
(3, 'Baburao', 'Apte', '2014-06-11', 'M', '1984-02-01')
(4, 'Anjali', 'Pande', '2000-08-23', 'F', '1978-05-19')
(5, 'Abhilasha', 'Mohite', '2020-03-30', 'F', '1994-01-17')
(6, 'Suresh', 'Kadam', '1999-02-21', 'M', '1977-08-21')
(7, 'Manish', 'Joshi', '2014-06-11', 'M', '1992-09-29')
(8, 'Radha', 'Marathe', '2020-03-29', 'F', '1988-11-22')


In [25]:
q2 = """
SELECT employees.first_name, employees.last_name FROM employees JOIN dept_manager ON employees.emp_no = dept_manager.emp_no;
"""
connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
results = read_query(connection, q2)

print(results)

MySQL Database connection successful
[('Shyam', 'Naik'), ('Baburao', 'Apte'), ('Suresh', 'Kadam'), ('Radha', 'Marathe')]


In [26]:
q3 = """
SELECT dept_emp.emp_no, employees.first_name, employees.last_name, dept_manager.dept_no, titles.title, salaries.from_date, salaries.to_date, salaries.salary FROM dept_emp  JOIN dept_manager ON dept_emp.emp_no = dept_manager.emp_no JOIN titles on dept_manager.emp_no = titles.emp_no JOIN employees ON employees.emp_no = dept_manager.emp_no JOIN salaries ON salaries.emp_no = dept_manager.emp_no"""

connection = create_db_connection("localhost", "root", 'mlwithrish30', 'company')
results = read_query(connection, q3)

print(results)

MySQL Database connection successful
[(2, 'Shyam', 'Naik', 'SFT01', 'Soft.Developer Mngr.', '2007-11-08', '2022-11-19', 2345678), (3, 'Baburao', 'Apte', 'MKT01', 'Marketing Mngr', '2014-06-11', '2022-11-19', 3456789), (6, 'Suresh', 'Kadam', 'SFT02', 'Soft.Tester Mngr', '1999-02-21', '2022-11-19', 6789012), (8, 'Radha', 'Marathe', 'HR01', 'HR Manager', '2020-03-29', '2022-11-19', 8901234)]


In [27]:
# Returns a list of lists and then creates a pandas DataFrame
import pandas as pd

columns = ["Employee_no", "First_name", "Last_name", "Department no.", "Job_role", "From_date", "To_date", "Salary"]
df = pd.DataFrame(results, columns=columns)

df.head()

,Employee_no,First_name,Last_name,Department no.,Job_role,From_date,To_date,Salary
0,2,Shyam,Naik,SFT01,Soft.Developer Mngr.,2007-11-08,2022-11-19,2345678
1,3,Baburao,Apte,MKT01,Marketing Mngr,2014-06-11,2022-11-19,3456789
2,6,Suresh,Kadam,SFT02,Soft.Tester Mngr,1999-02-21,2022-11-19,6789012
3,8,Radha,Marathe,HR01,HR Manager,2020-03-29,2022-11-19,8901234


With just a few lines of code, we can easily extract all the data we can handle from the relational databases where it lives, and pull it into our state-of-the-art data analytics pipelines. This is really helpful stuff.

